#### Initialise environment

In [3]:
import os
from dotenv import load_dotenv
load_dotenv('../.env', override=True)

True

In [4]:
# define the Azure AI Studio connection
azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
    #"credential": token_provider,
}

### load the LLM wrapper

In [7]:
# callable function that invokes Azure OpenAI.  For use as target in evaluator.
from genai.llm import llm_tool

# Azure Content Safety-based Protected Material evaluator

### Protected Material simulator

In [8]:
from azure.identity import DefaultAzureCredential
from promptflow.evals.synthetic import AdversarialSimulator, AdversarialScenario

protected_material_scenario = AdversarialScenario.PROTECTED_MATERIAL_CHAT

simulator = AdversarialSimulator(azure_ai_project=azure_ai_project, credentials=DefaultAzureCredential())
outputs = await simulator(
        scenario=protected_material_scenario, # required, adversarial scenario to simulate
        target=llm_tool, # required, callback function to simulate against
        max_conversation_turns=2, # optional
        max_simulation_results=3, # optional, default 3
    )


AttributeError: type object 'AdversarialScenario' has no attribute 'PROTECTED_MATERIAL_CHAT'

### Indirect Attack (XPIA)

In [ ]:
from promptflow.evals.synthetic import IndirectAttackSimulator, DirectAttackSimulator, AdversarialScenario

XPIAsimulator = IndirectAttackSimulator(azure_ai_project=azure_ai_project)
UPIAsimulator = DirectAttackSimulator(azure_ai_project=azure_ai_project) # the current jailbreak simulator will get taken out of the AdversarialSimulator into its own class

outputs = await XPIAsimulator(
        target=callback, # required, callback function to simulate against
        max_conversation_turns=1, #optional, 
        max_simulation_results=3 #optional
    )

In [ ]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    query = messages["messages"][0]["content"]
    # the XP injection is in the context
    context = context['xpia_injection_text']

    # Add file contents for summarization or re-write
    if 'file_content' in messages["template_parameters"]:
        query += messages["template_parameters"]['file_content']
    
    # Call your own endpoint and pass your query as input. Make sure to handle your function_call_to_your_endpoint's error responses.
    # the XP injection is in the context, make sure your application uses the context to construct its response.
    response = await function_call_to_your_endpoint(query, context) 
    
    # Format responses in OpenAI message protocol
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {},
    }

    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context
    }

## Generate AI-simulated non-adversarial dataset

In [ ]:
import asyncio
import json
import os
from typing import Any, Dict, List, Optional

import wikipedia
from azure.identity import DefaultAzureCredential

from promptflow.evals.synthetic import Simulator

wiki_search_term = "Leonardo da vinci"
wiki_title = wikipedia.search(wiki_search_term)[0]
wiki_page = wikipedia.page(wiki_title)
text = wiki_page.summary[:5000]


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_your_endpoint_here(query=query, context=context, conversation_history=messages_list)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}



async def main(azure_ai_project):
    simulator = Simulator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
    outputs = await simulator(
        target=callback, # required
        text=text, # required
        num_queries=4, # optional, defaults to len (tasks)
        max_conversation_turns=2, # optional, defaults to 2
        tasks=[
            f"I am a student and I want to learn more about {wiki_search_term}",
            f"I am a teacher and I want to teach my students about {wiki_search_term}",
            f"I am a researcher and I want to do a detailed research on {wiki_search_term}",
            f"I am a statistician and I want to do a detailed table of factual data concerning {wiki_search_term}",
        ],
    )

    for output in outputs:
        with open("output.jsonl", "a") as f:
            f.write(output.to_eval_qa_json_lines())



if __name__ == "__main__":
    os.environ["AZURE_SUBSCRIPTION_ID"] = ""
    os.environ["RESOURCE_GROUP"] = ""
    os.environ["PROJECT_NAME"] = ""
    os.environ["AZURE_OPENAI_API_KEY"] = ""
    os.environ["AZURE_OPENAI_ENDPOINT"] = ""
    os.environ["AZURE_DEPLOYMENT"] = ""

    azure_ai_project = {
        "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
        "resource_group_name": os.environ.get("RESOURCE_GROUP"),
        "project_name": os.environ.get("PROJECT_NAME"),
    }

    asyncio.run(main(azure_ai_project=azure_ai_project))
    print("done!")

In [10]:
from promptflow.evals.synthetic import Simulator

simulator = Simulator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
outputs = await simulator(
    target=llm_tool, # required
    text=text, # required
    num_queries=4, # required
    max_conversation_turns=2, # optional, defaults to 2
)

ImportError: cannot import name 'Simulator' from 'promptflow.evals.synthetic' (/home/krbock/miniconda3/envs/pf-eval/lib/python3.12/site-packages/promptflow/evals/synthetic/__init__.py)